In [246]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import os
import json
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

In [247]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [248]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [249]:
analyzer = SentimentIntensityAnalyzer

In [282]:
drive_path = '/content/drive/MyDrive/data/disab/2010s/json'

In [283]:
data = []

In [284]:
from pathlib import Path
json_path = Path(drive_path)
json_paths = list(json_path.rglob('*.json'))
print(f'Found {len(json_paths)} json files')

Found 5 json files


In [285]:
bodies = []
for f in json_paths:
  data = json.load(f.open())
  for d in data:
    bodies.append(d['body'])
print(f'Found {len(bodies)} bodies')

Found 499 bodies


In [286]:
type(bodies)

list

In [287]:
sia = SentimentIntensityAnalyzer()
sents = [sia.polarity_scores(' '.join(b)) for b in bodies]
df = pd.DataFrame(sents)
df

,neg,neu,pos,compound
0,0.012,0.903,0.084,0.9645
1,0.147,0.768,0.085,-0.9968
2,0.028,0.840,0.132,0.9972
3,0.021,0.829,0.150,0.9976
4,0.000,0.831,0.169,0.9881
...,...,...,...,...
494,0.039,0.885,0.076,0.9999
495,0.029,0.825,0.146,0.9976
496,0.048,0.817,0.135,0.9961
497,0.006,0.865,0.130,0.9867


In [288]:
sentiment = df['compound']
threshold = 0.05
df['Sentiment Label'] = df['compound'].apply(lambda x: 1 if x >= threshold else 0)
sortedDataFrame = df.sort_values(by='compound', ascending=True)
sortedDataFrame

,neg,neu,pos,compound,Sentiment Label
150,0.123,0.805,0.071,-0.9990,0
297,0.170,0.745,0.085,-0.9990,0
199,0.283,0.668,0.048,-0.9989,0
424,0.245,0.693,0.062,-0.9988,0
200,0.159,0.772,0.069,-0.9988,0
...,...,...,...,...,...
116,0.071,0.796,0.132,0.9999,1
287,0.061,0.815,0.124,0.9999,1
446,0.029,0.844,0.127,0.9999,1
389,0.048,0.848,0.104,1.0000,1


In [289]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(10000, 128, input_length=300))
model.add(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [290]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [298]:
x_train, x_test, y_train, y_test = train_test_split(sentiment, df["Sentiment Label"], test_size=0.8, random_state=42)

In [300]:
history = model.fit(x_train, y_train, batch_size=55, epochs=5, validation_split=0.2)

Epoch 1/5
2/2 [==============================] - 0s 151ms/step - loss: 0.3581 - accuracy: 0.8861 - val_loss: 0.3254 - val_accuracy: 0.9000
Epoch 2/5
2/2 [==============================] - 0s 49ms/step - loss: 0.3559 - accuracy: 0.8861 - val_loss: 0.3255 - val_accuracy: 0.9000
Epoch 3/5
2/2 [==============================] - 0s 70ms/step - loss: 0.3576 - accuracy: 0.8861 - val_loss: 0.3259 - val_accuracy: 0.9000
Epoch 4/5
2/2 [==============================] - 0s 52ms/step - loss: 0.3573 - accuracy: 0.8861 - val_loss: 0.3266 - val_accuracy: 0.9000
Epoch 5/5
2/2 [==============================] - 0s 53ms/step - loss: 0.3584 - accuracy: 0.8861 - val_loss: 0.3269 - val_accuracy: 0.9000


In [301]:
loss, accuracy = model.evaluate(x_test, y_test)

13/13 [==============================] - 0s 3ms/step - loss: 0.4319 - accuracy: 0.8475


In [302]:
sentimentPrediction = model.predict(sentiment)

16/16 [==============================] - 0s 2ms/step
